In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


In [2]:
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
                              

In [3]:
train_data=torchvision.datasets.CIFAR10(root='./data',train=True,transform=transform,download=True)
test_data=torchvision.datasets.CIFAR10(root='./data',train=False,transform=transform,download=True)

train_loader=torch.utils.data.DataLoader(train_data,batch_size=32,shuffle=True,num_workers=2)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=32,shuffle=True,num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
image,label=train_data[0]

In [5]:
class_names=['plane','car','cat','dog','deer','dog','frog','horse','ship','truck']

In [6]:
class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__() #to call parent class within child class

        self.conv1=nn.Conv2d(3,12,5) #12,28,28
        self.pool=nn.MaxPool2d(2,2)#12,14,14
        self.conv2=nn.Conv2d(12,24,5) #24,10,10
        self.fc1=nn.Linear(24*5*5,120)
        self.fc2=nn.Linear(120,70)
        self.fc3=nn.Linear(70,10)
        
    def forward(self,x):
       x=self.pool(F.relu(self.conv1(x)))   #max sizing or avg sizing is self.pool #F.relu is nonlinearity  # self.conv1 is to applying cnn to extract features of x  
       x=self.pool(F.relu(self.conv2(x))) 
       x= torch.flatten(x,1)  
       x=F.relu(self.fc1(x))     
       x=F.relu(self.fc2(x))  # to extract or identify pattern 
       x=self.fc3(x)
       return x

In [7]:
net=NeuralNet()
loss_function=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=0.001,momentum=0.9)


In [8]:
for epoch in range(32):
  running_loss=0.0
  print(f"Training epoch {epoch}")
  for i,data in enumerate (train_loader):
      inputs,label=data
      optimizer.zero_grad()
      output=net(inputs)
      loss=loss_function(output,label)
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()

  print(f"loss: {running_loss/len(train_loader)}:.4f")

Training epoch 0
loss: 2.193670572566437:.4f
Training epoch 1
loss: 1.7097747618772248:.4f
Training epoch 2


KeyboardInterrupt: 

In [14]:
torch.save(net.state_dict(),'trained_net.pth')

In [15]:
net.load_state_dict(torch.load('trained_net.pth'))

C:\Users\HP LAPTOP\AppData\Local\Temp\ipykernel_16400\3199039750.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [18]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 60000 test images: %d %%' % (
    100 * correct / total))         

Accuracy of the network on the 60000 test images: 41 %


In [24]:
new_transform=transforms.Compose([transforms.Resize((32,32)),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

def load_image(image_path):
    image=Image.open(image_path)
    image=new_transform(image)
    image=image.unsqueeze(0)
    return image

image_paths=['ex1.jfif','example2.jfif']
images=[load_image(image) for image in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        outputs=net(image)
        _, predicted = torch.max(outputs.data, 1)
        print(f'Prediction : {class_names[predicted.item()]}')

Prediction : plane
Prediction : car
